In [ ]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

In [ ]:
sys.path.append('../..')

In [ ]:
from lung_cancer.dataset import FilesIndex, Pipeline, Dataset
from lung_cancer import CTImagesMaskedBatch as CTIMB
from lung_cancer import combine_crops

### Set up the two datasets

* indices

In [ ]:
DIR_CANCER = '/data/lunaset_split/train/cancer/*'
DIR_NCANCER = '/data/lunaset_split/train/ncancer/*'

In [ ]:
cancerix = FilesIndex(path=DIR_CANCER, dirs=True)

In [ ]:
ncancerix = FilesIndex(path=DIR_NCANCER, dirs=True)

* datasets

In [ ]:
cancerset = Dataset(index=cancerix, batch_class=CTIMB)
ncancerset = Dataset(index=ncancerix, batch_class=CTIMB)

### Pipeline

In [ ]:
help(combine_crops)

In [ ]:
# we want a pipeline, that will generate batch with 20 non-cancerous nodules
# and 20 cancerous
ppl = combine_crops(cancerset, ncancerset, batch_sizes=(20, 20))

In [ ]:
# generate a batch 
# Note that next_batch'ing is performed without args
bch = ppl.next_batch()

In [ ]:
# 40 = 20 + 20
len(bch)

In [ ]:
# first 20 items are non-cancerous
# (though it is possible that a bit of cancer will be on some crops)
np.sum(bch.get(7, 'masks'))

In [ ]:
np.sum(bch.get(17, 'masks'))

In [ ]:
# while the other items are quite cancerous
np.sum(bch.get(27, 'masks'))

In [ ]:
np.sum(bch.get(37, 'masks'))

### Test generated batches

In [ ]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [ ]:
interact(lambda height: plot_arr_slices(height, 
                                        bch.get(21, 'images'),
                                        bch.get(21, 'masks'), clim=(0, 255)),
         height=(0.01, 0.99, 0.01))